<a href="https://colab.research.google.com/github/rickiepark/llm-from-scratch/blob/main/ch05/01_main-chapter-code/exercise-solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
세바스찬 라시카(Sebastian Raschka)가 쓴 <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a>의 번역서 예제 코드입니다.<br>
<br>코드 저장소: <a href="https://github.com/rickiepark/llm-from-scratch">https://github.com/rickiepark/llm-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>

# 5장 연습문제 솔루션


In [ ]:
from importlib.metadata import version

pkgs = ["numpy",
        "tiktoken",
        "torch",
        "tensorflow" # OpenAI의 사전 훈련된 가중치를 위해
       ]
for p in pkgs:
    print(f"{p} 버전: {version(p)}")

# 연습문제 5.1: 온도 스케일링이 적용된 소프트맥스 점수 및 샘플링 확률


- 이 절에서 정의한 `print_sampled_tokens` 함수를 사용하여 "pizza" 단어가 샘플링된 횟수를 출력할 수 있습니다.
- 5.3.1 섹션에서 정의한 코드부터 시작하겠습니다.

- 온도가 0 또는 0.1인 경우 0회 샘플링되고, 온도가 5이면 32회 샘플링됩니다. 추정 확률은 32/1000 * 100% = 3.2%입니다.

- 실제 확률은 4.3%이며, 스케일이 조정된 소프트맥스 확률 텐서(`scaled_probas[2][6]`)에 포함되어 있습니다.


In [ ]:
import torch

vocab = {
    "closer": 0,
    "every": 1,
    "effort": 2,
    "forward": 3,
    "inches": 4,
    "moves": 5,
    "pizza": 6,
    "toward": 7,
    "you": 8,
}
inverse_vocab = {v: k for k, v in vocab.items()}

next_token_logits = torch.tensor(
    [4.51, 0.89, -1.90, 6.75, 1.63, -1.62, -1.89, 6.28, 1.79]
)

def print_sampled_tokens(probas):
    torch.manual_seed(123)
    sample = [torch.multinomial(probas, num_samples=1).item() for i in range(1_000)]
    sampled_ids = torch.bincount(torch.tensor(sample))
    for i, freq in enumerate(sampled_ids):
        print(f"{freq} x {inverse_vocab[i]}")


def softmax_with_temperature(logits, temperature):
    scaled_logits = logits / temperature
    return torch.softmax(scaled_logits, dim=0)


temperatures = [1, 0.1, 5]  # 원본, 높은, 그리고 낮은 온도
scaled_probas = [softmax_with_temperature(next_token_logits, T) for T in temperatures]

- 이제 `scaled_probas`를 반복하면서 샘플링 빈도를 출력할 수 있습니다:


In [ ]:
for i, probas in enumerate(scaled_probas):
    print("\n\n온도:", temperatures[i])
    print_sampled_tokens(probas)

- "pizza"라는 단어가 샘플링될 때 샘플링은 실제 확률의 근사치를 제공한다는 점에 유의하세요.
- 예: 1000번 중 32번 샘플링된 경우 추정 확률은 3.2%입니다.
- 실제 확률을 얻으려면 `scaled_probas`에서 해당 항목에 직접 액세스하여 확률을 확인할 수 있습니다.

- "pizza"는 어휘에서 7번째 항목이므로 온도 5의 경우 다음과 같이 얻습니다.

In [ ]:
temp5_idx = 2
pizza_idx = 6

scaled_probas[temp5_idx][pizza_idx]

온도가 5로 설정된 경우 "pizza"라는 단어가 샘플링될 확률은 4.3%입니다.


# 연습문제 5.2: 다양한 온도 및 top-k 설정

- 온도 및 top-k 설정 모두 개별 LLM에 따라 조정해야 합니다 (원하는 출력을 생성할 때까지 시행착오 과정).
- 바람직한 결과는 애플리케이션마다 다릅니다.
  - 낮은 top-k 및 온도는 교육 콘텐츠, 기술 문서 또는 질문 응답, 데이터 분석, 코드 생성 등을 작성할 때 바람직한 덜 무작위적인 결과를 생성합니다.
  - 높은 top-k 및 온도는 브레인스토밍 작업, 창의적인 글쓰기 등에 더 바람직한 더 다양하고 무작위적인 출력을 생성합니다.


# 연습문제 5.3: 디코딩 함수의 결정론적인 동작


`generate` 함수에서 결정론적인 동작을 강제하는 방법은 여러 가지가 있습니다:

1. `temperature=0.0`으로 설정;
2. `top_k=1`로 설정.


In [ ]:
import tiktoken
import torch
from previous_chapters import GPTModel


GPT_CONFIG_124M = {
    "vocab_size": 50257,    # 어휘 크기
    "context_length": 256,  # 짧아진 컨텍스트 길이 (원래: 1024)
    "emb_dim": 768,         # 임베딩 차원
    "n_heads": 12,          # 어텐션 헤드 수
    "n_layers": 12,         # 층 수
    "drop_rate": 0.1,       # 드롭아웃 비율
    "qkv_bias": False       # 쿼리-키-값 편향
}


torch.manual_seed(123)

tokenizer = tiktoken.get_encoding("gpt2")
model = GPTModel(GPT_CONFIG_124M)
model.load_state_dict(torch.load("model.pth", weights_only=True))
model.eval();

In [ ]:
from gpt_generate import generate, text_to_token_ids, token_ids_to_text
from previous_chapters import generate_text_simple

In [ ]:
# torch.argmax를 사용하는 결정론적 함수

start_context = "Every effort moves you"

token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids(start_context, tokenizer),
    max_new_tokens=25,
    context_size=GPT_CONFIG_124M["context_length"]
)

print("출력 텍스트:\n", token_ids_to_text(token_ids, tokenizer))

In [ ]:
# 결정론적 동작: top_k 없음, 온도 스케일링 없음

token_ids = generate(
    model=model,
    idx=text_to_token_ids("Every effort moves you", tokenizer),
    max_new_tokens=25,
    context_size=GPT_CONFIG_124M["context_length"],
    top_k=None,
    temperature=0.0
)

print("출력 텍스트:\n", token_ids_to_text(token_ids, tokenizer))

- 이전 코드 셀을 다시 실행하면 똑같은 텍스트가 생성됩니다.


In [ ]:
# 결정론적 동작: top_k 없음, 온도 스케일링 없음

token_ids = generate(
    model=model,
    idx=text_to_token_ids("Every effort moves you", tokenizer),
    max_new_tokens=25,
    context_size=GPT_CONFIG_124M["context_length"],
    top_k=None,
    temperature=0.0
)

print("출력 텍스트:\n", token_ids_to_text(token_ids, tokenizer))

# 연습문제 5.4: 사전 훈련 계속 수행


- 5장에서 모델을 처음 훈련했던 파이썬 세션에 아직 있다면, 사전 훈련을 한 에포크 더 계속하려면 이 장의 노트북에서 저장한 모델과 옵티마이저를 로드하고 `train_model_simple` 함수를 다시 호출하면 됩니다.

- 이 새로운 코드 환경에서 재현가능하도록 만들려면 몇 가지 단계가 더 필요합니다.
- 먼저 토크나이저, 모델 및 옵티마이저를 로드합니다.


In [ ]:
import tiktoken
import torch
from previous_chapters import GPTModel


GPT_CONFIG_124M = {
    "vocab_size": 50257,   # 어휘 크기
    "context_length": 256, # 짧아진 컨텍스트 길이 (원래: 1024)
    "emb_dim": 768,        # 임베딩 차원
    "n_heads": 12,         # 어텐션 헤드 수
    "n_layers": 12,        # 층 수
    "drop_rate": 0.1,      # 드롭아웃 비율
    "qkv_bias": False      # 쿼리-키-값 편향
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = tiktoken.get_encoding("gpt2")

checkpoint = torch.load("model_and_optimizer.pth", weights_only=True)
model = GPTModel(GPT_CONFIG_124M)
model.load_state_dict(checkpoint["model_state_dict"])
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=0.0004, weight_decay=0.1)
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
model.train();

- 다음으로, 데이터 로더를 초기화합니다:


In [ ]:
import os
import urllib.request
from previous_chapters import create_dataloader_v1


file_path = "the-verdict.txt"
url = "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/main/ch02/01_main-chapter-code/the-verdict.txt"

if not os.path.exists(file_path):
    with urllib.request.urlopen(url) as response:
        text_data = response.read().decode('utf-8')
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(text_data)
else:
    with open(file_path, "r", encoding="utf-8") as file:
        text_data = file.read()


# 훈련/검증 비율
train_ratio = 0.90
split_idx = int(train_ratio * len(text_data))
train_data = text_data[:split_idx]
val_data = text_data[split_idx:]


torch.manual_seed(123)

train_loader = create_dataloader_v1(
    train_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=True,
    shuffle=True,
    num_workers=0
)

val_loader = create_dataloader_v1(
    val_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=False,
    shuffle=False,
    num_workers=0
)

- 마지막으로, `train_model_simple` 함수를 사용하여 모델을 학습시킵니다:


In [ ]:
from gpt_train import train_model_simple

num_epochs = 1
train_losses, val_losses, tokens_seen = train_model_simple(
    model, train_loader, val_loader, optimizer, device,
    num_epochs=num_epochs, eval_freq=5, eval_iter=5,
    start_context="Every effort moves you", tokenizer=tokenizer
)

# 연습문제 5.5: 사전 훈련된 모델의 훈련 및 검증 세트 손실


- 다음 코드를 사용하여 GPT 모델의 훈련 및 검증 세트 손실을 계산할 수 있습니다:

```python
train_loss = calc_loss_loader(train_loader, gpt, device)
val_loss = calc_loss_loader(val_loader, gpt, device)
```

- 124M 매개변수에 대한 결과 손실은 다음과 같습니다:

```
Training loss: 3.754748503367106
Validation loss: 3.559617757797241
```

- 주요 관찰 사항은 학습 세트와 검증 세트의 성능이 비슷하다는 것입니다.
- 이는 여러 가지 이유로 설명될 수 있습니다:

1. OpenAI가 GPT-2를 훈련시킬 때 The Verdict는 사전 훈련 데이터 세트에 포함되지 않았습니다. 따라서 모델은 훈련 세트에 명시적으로 과적합되지 않으며 The Verdict의 훈련 및 검증 세트 부분에서 유사하게 잘 수행됩니다. (검증 세트 손실이 훈련 세트 손실보다 약간 낮은데, 이는 딥러닝에서는 드문 현상입니다. 하지만 데이터 세트가 비교적 작기 때문에 무작위 잡음 때문일 가능성이 큽니다. 실제로 과대적합이 없다면 훈련 세트와 검증 세트의 성능은 거의 동일할 것으로 예상됩니다).

2. The Verdict가 GPT-2의 훈련 데이터셋에 포함되었습니다. 이 경우 검증 세트도 학습에 사용되었을 것이기 때문에 모델이 훈련 데이터에 과대적합되었는지 여부를 알 수 없습니다. 과대적합 정도를 평가하려면 OpenAI가 GPT-2 학습을 완료한 후 생성된 새로운 데이터셋가 필요합니다. 이는 사전 학습에 사용되지 않았음을 확인하기 위한 것입니다.


In [ ]:
import tiktoken
import torch
from previous_chapters import GPTModel


GPT_CONFIG_124M = {
    "vocab_size": 50257,   # 어휘 크기
    "context_length": 256, # 짧아진 컨텍스트 길이 (원래: 1024)
    "emb_dim": 768,        # 임베딩 차원
    "n_heads": 12,         # 어텐션 헤드 수
    "n_layers": 12,        # 레이어 수
    "drop_rate": 0.1,      # 드롭아웃 비율
    "qkv_bias": False      # 쿼리-키-값 편향
}


torch.manual_seed(123)

tokenizer = tiktoken.get_encoding("gpt2")

In [ ]:
from gpt_download import download_and_load_gpt2

settings, params = download_and_load_gpt2(model_size="124M", models_dir="gpt2")

In [ ]:
# 모델 구성을 간결하게 딕셔너리로 정의합니다.
model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

# 기본 구성을 복사하고 특정 모델 설정으로 업데이트합니다.
model_name = "gpt2-small (124M)"  # 예시 모델 이름
NEW_CONFIG = GPT_CONFIG_124M.copy()
NEW_CONFIG.update(model_configs[model_name])
NEW_CONFIG.update({"context_length": 1024, "qkv_bias": True})

gpt = GPTModel(NEW_CONFIG)
gpt.eval();

In [ ]:
from gpt_generate import load_weights_into_gpt


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
load_weights_into_gpt(gpt, params)
gpt.to(device);

In [ ]:
import os
import urllib.request
from previous_chapters import create_dataloader_v1


file_path = "the-verdict.txt"
url = "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/main/ch02/01_main-chapter-code/the-verdict.txt"

if not os.path.exists(file_path):
    with urllib.request.urlopen(url) as response:
        text_data = response.read().decode('utf-8')
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(text_data)
else:
    with open(file_path, "r", encoding="utf-8") as file:
        text_data = file.read()


# 훈련/검증 비율
train_ratio = 0.90
split_idx = int(train_ratio * len(text_data))
train_data = text_data[:split_idx]
val_data = text_data[split_idx:]


torch.manual_seed(123)

train_loader = create_dataloader_v1(
    train_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=True,
    shuffle=True,
    num_workers=0
)

val_loader = create_dataloader_v1(
    val_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=False,
    shuffle=False,
    num_workers=0
)

In [ ]:
from gpt_train import calc_loss_loader

torch.manual_seed(123) # 데이터 로더에서 셔플링을 하므로 재현성을 위해
train_loss = calc_loss_loader(train_loader, gpt, device)
val_loss = calc_loss_loader(val_loader, gpt, device)

print("훈련 손실:", train_loss)
print("검증 손실:", val_loss)

가장 큰 GPT-2 모델에서도 이를 반복할 수 있지만, 컨텍스트 길이를 업데이트하는 것을 잊지 마세요:


In [ ]:
settings, params = download_and_load_gpt2(model_size="1558M", models_dir="gpt2")

model_name = "gpt2-xl (1558M)"
NEW_CONFIG = GPT_CONFIG_124M.copy()
NEW_CONFIG.update(model_configs[model_name])
NEW_CONFIG.update({"context_length": 1024, "qkv_bias": True})

gpt = GPTModel(NEW_CONFIG)
gpt.eval()

load_weights_into_gpt(gpt, params)
gpt.to(device)

torch.manual_seed(123)
train_loss = calc_loss_loader(train_loader, gpt, device)
val_loss = calc_loss_loader(val_loader, gpt, device)

print("훈련 손실:", train_loss)
print("검증 손실:", val_loss)

# 연습문제 5.6: 더 큰 모델 시도하기


- 메인 챕터에서는 124M 매개변수만 있는 가장 작은 GPT-2 모델을 사용하여 실험했습니다.
- 이유는 리소스 요구 사항을 최대한 낮추기 위해서였습니다.
- 그러나 최소한의 코드 변경으로 더 큰 모델을 쉽게 실험할 수 있습니다.
- 예를 들어, 5장에서 124M 모델 대신 1558M 모델을 로드하는 경우 변경해야 하는 코드는 두 줄뿐입니다.

```python
settings, params = download_and_load_gpt2(model_size="124M", models_dir="gpt2")
model_name = "gpt2-small (124M)"
```

- 업데이트된 코드는 다음과 같습니다.


```python
settings, params = download_and_load_gpt2(model_size="1558M", models_dir="gpt2")
model_name = "gpt2-xl (1558M)"
```


In [ ]:
import tiktoken
import torch
from previous_chapters import GPTModel


GPT_CONFIG_124M = {
    "vocab_size": 50257,   # 어휘 크기
    "context_length": 256, # 단축된 컨텍스트 길이 (원래: 1024)
    "emb_dim": 768,        # 임베딩 차원
    "n_heads": 12,         # 어텐션 헤드 수
    "n_layers": 12,        # 층 수
    "drop_rate": 0.1,      # 드롭아웃 비율
    "qkv_bias": False      # 쿼리-키-값 편향
}


tokenizer = tiktoken.get_encoding("gpt2")

In [ ]:
from gpt_download import download_and_load_gpt2
from gpt_generate import load_weights_into_gpt


model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

model_name = "gpt2-xl (1558M)"
NEW_CONFIG = GPT_CONFIG_124M.copy()
NEW_CONFIG.update(model_configs[model_name])
NEW_CONFIG.update({"context_length": 1024, "qkv_bias": True})

gpt = GPTModel(NEW_CONFIG)
gpt.eval()

settings, params = download_and_load_gpt2(model_size="1558M", models_dir="gpt2")
load_weights_into_gpt(gpt, params)

In [ ]:
from gpt_generate import generate, text_to_token_ids, token_ids_to_text

In [ ]:
torch.manual_seed(123)

token_ids = generate(
    model=gpt,
    idx=text_to_token_ids("Every effort moves you", tokenizer),
    max_new_tokens=25,
    context_size=NEW_CONFIG["context_length"],
    top_k=50,
    temperature=1.5
)

print("출력 텍스트:\n", token_ids_to_text(token_ids, tokenizer))